In [124]:
import numpy as np
import pandas as pd

In [125]:
df = pd.read_excel('data/merged_df.xlsx')
df = df[['occupation', 'vacancyTitle']].rename(columns={'occupation': 'labels', 'vacancyTitle': 'title'})
df.head()

,labels,title
0,marketing assistant,Marketing
1,marketing assistant,Marketing Advisor
2,marketing assistant,Admin Marketing
3,marketing assistant,SALES EXECUTIVE MANAGER
4,marketing assistant,Talent Executive


In [126]:
def clean_str(s: str) -> str:
    s = s.lower()
    return s
df.loc[:, 'title'] = df.title.apply(clean_str)

In [127]:

df.loc[:, 'title'] = df.title.apply(clean_str)
label_to_idx = {l: i for i, l in enumerate(df.labels.unique())}
idx_to_label = {i: l for i, l in enumerate(df.labels.unique())}

# change the label to number
df.loc[:, 'labels'] = df.labels.apply(lambda x: label_to_idx.get(x))

len(idx_to_label)
df

,labels,title
0,0,marketing
1,0,marketing advisor
2,0,admin marketing
3,0,sales executive manager
4,0,talent executive
...,...,...
15458,1364,transfer harvest finished goods supervisor
15459,1365,treasury intern
15460,1366,meat & vegetables executives
15461,1367,site supervisor for water pipe works (location...


In [128]:
# import numpy as np
# v
# from torch.utils.data import DataLoader
# data = df.title
# label = df.labels
# ds = Dataset.from_dict({"data": data, "label": label}).with_format("torch")
# dataloader = DataLoader(ds, batch_size=4)
# for batch in dataloader:
#     print(batch)

In [129]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [130]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["title"], padding='max_length', max_length = 64, truncation=True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\itama/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.11.3",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\itama/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee7

In [131]:
tokenized_data = dataset.map(preprocess_function)
tokenized_data = tokenized_data.remove_columns(["title"])
tokenized_data.set_format("torch")


  0%|          | 0/15463 [00:00<?, ?ex/s]

In [132]:
tokenized_data[3]['input_ids']

tensor([ 101, 4341, 3237, 3208,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [133]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_data, shuffle=True, batch_size=8)
for batch in train_dataloader:
    print(batch)
    break

{'labels': tensor([104,  57,  90, 400, 142, 195,  64, 134]), 'input_ids': tensor([[  101, 25718, 13679, 15985, 28547,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101, 17850,  2961,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,    

In [134]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [135]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(idx_to_label))

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\itama/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL

In [136]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [137]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [138]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/5799 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [139]:
output = tokenizer.decode(tokenized_data[3]['input_ids'].squeeze(), skip_special_tokens=True)
output

'sales executive manager'

In [140]:
# metric = load_metric("accuracy")
model.eval()
for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    print(logits.shape)
    o = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)
    la = batch['labels'].tolist()
    predictions = torch.argmax(logits, dim=-1)
    pred = predictions.tolist()
    for t, p, l in zip(o, pred, la):
        print(f'title: {t},\n label: {idx_to_label.get(p)}, \n real: {idx_to_label.get(l)}')
        print('\n')
    print('--------------')
#     metric.add_batch(predictions=predictions, references=batch["labels"])
#
# metric.compute()

torch.Size([8, 1369])
title: senior java developer / architect,
 label: ethical hacker, 
 real: ICT system developer


title: data scientist,
 label: ethical hacker, 
 real: data scientist


title: cleaner office,
 label: ethical hacker, 
 real: building cleaner


title: assistant supervisor,
 label: financial markets back office administrator, 
 real: office manager


title: student support and pathway coordinator,
 label: ethical hacker, 
 real: academic support officer


title: compressor commissioning specialist,
 label: ethical hacker, 
 real: rotating equipment mechanic


title: officer, supply chain service verification ( korean speaker ),
 label: investment fund management assistant, 
 real: supply chain assistant


title: content review management ( zulu market ),
 label: ethical hacker, 
 real: online community manager


--------------
torch.Size([8, 1369])
title: digital marketing executives,
 label: ethical hacker, 
 real: online sales channel manager


title: sales enginee

KeyboardInterrupt: 

idx_to_label